**Load utils and lib**

In [52]:
import pandas as pd
import numpy as np
import math
import string
import os, sys
import json

In [2]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 5.1 MB/s 
     |████████████████████████████████| 6.5 MB 34.4 MB/s 
     |████████████████████████████████| 895 kB 57.6 MB/s 
     |████████████████████████████████| 67 kB 4.9 MB/s 
     |████████████████████████████████| 596 kB 54.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [12]:
!pip install sentencepiece

In [1]:
# after installing sentencepiece, need to restart kernel
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, T5Tokenizer
from transformers import logging as transformers_logging
tokenizer = AutoTokenizer.from_pretrained("allenai/macaw-large")
model = AutoModelForSeq2SeqLM.from_pretrained("allenai/macaw-large")
input_string = "$answer$ ; $mcoptions$ ; $question$ = What is the color of a cloudy sky?"
input_ids = tokenizer.encode(input_string, return_tensors="pt")
output = model.generate(input_ids, max_length=200)

tokenizer.batch_decode(output, skip_special_tokens=True)

Downloading:   0%|          | 0.00/2.75G [00:00<?, ?B/s]

['$answer$ = gray ; $mcoptions$ = (A) blue (B) white (C) grey (D) white']

In [ ]:
# first upload utils.py to /content/
import utils
from utils import load_model,run_macaw

model_dict = load_model("allenai/macaw-large", cuda_devices=None) 

In [3]:
print(utils)

<module 'utils' from '/content/utils.py'>


**Experiment trial**

In [4]:
res = run_macaw("Q: James went camping in the woods, but forgot to bring a hammer to bang the tent pegs in. What else might he use?\nA", model_dict)
res['output_slots_list'][0]

{'answer': 'a chisel'}

In [5]:
# Add more reqturn sequences and make the outputs less repetative
good = run_macaw("Q: What dose speaker imply when he says that Alex is a good teacher?\nA\nE", model_dict, 
{"do_sample": True, "top_k": 10, "num_return_sequences": 5, "repetition_penalty": 3.0})

good['output_slots_list']

[{'answer': 'the speaker is implying that alex has much to teach others through example and communication with them, not just from his own experience as an athlete but also from being around other athletes.'},
 {'answer': "the speaker is implying that alex has good knowledge of math and science, but doesn't know much about language arts or education."},
 {'answer': "that he has high expectations for his students and does not expect much from them, but is very kind and patient with her student's needs. this statement implies the speaker think Alex can do well in school because of how hard she worked to make it happen so quickly! these statements suggest an assumption made by the speaker which may be incorrect? (A) That teacher knows many things about math; #explanation#: If something requires something else then that something allows that something else."},
 {'answer': 'he thinks alex is smart, but not very smart.'},
 {'answer': 'the speaker is implying that alex has a good teaching sty

**Define functions**

In [28]:
df = pd.read_csv('si_adj.csv')
df.head()

,condtion,target_adj,prediction,question,context,answer
0,SI,good,not excellent,What does the speaker imply by saying Alex is ...,The speaker is pragmatically savvy.,NaN
1,SI,warm,not hot; not cold,What does the speaker imply by saying it's war...,The speaker is pragmatically savvy.,NaN


In [41]:
def question(question_str, context = ""):
  res = run_macaw("Q: "+question_str+"\nC: "+context+" \nA", model_dict,
                  {"do_sample": True, 'max_length': 100,
                   "top_k": 5, "num_return_sequences": 5, "repetition_penalty": 3.0})
  return res

In [17]:
res1 = question("What does the speaker imply by saying Alex ate some of the cookies?", 
                "The speaker is pragmatically savvy.")

{'input_raw': '$answer$ ; $explanation$ ; $question$ = What does the speaker imply by saying Alex ate some of the cookies? ; $context$ = The speaker is pragmatically savvy.', 'output_raw_list': ['$answer$ = ate the cookies', '$answer$ = he must have enjoyed them. dessert tasted good, though it was cold outside; implies that the cookie had some nutritional value to its owner when she baked it', '$answer$ = was a cheater and stole the cookies from his friend', '$answer$ = he overate them and put some in his mouth to finish the sentence.', "$answer$ = the cookies were harmed by his mother's carelessness and negligence, which led to him not being able to finish them all. rather than sweep up their footprints in an accident"], 'generator_options': {'min_length': 1, 'max_length': 128, 'num_beams': 1, 'num_return_sequences': 5, 'do_sample': True, 'top_k': 10, 'top_p': 1.0, 'temperature': 1.0, 'length_penalty': 1.0, 'repetition_penalty': 3.0}, 'requested_angle': 'QC->AE', 'input_slots': {'Ques

In [ ]:
'''
{'input_raw': '$answer$ ; $explanation$ ; 
$question$ = What does the speaker imply by saying Alex ate some of the cookies? ; 
$context$ = The speaker is pragmatically savvy.', 
'output_raw_list': 
['$answer$ = ate the cookies', 
'$answer$ = he must have enjoyed them. dessert tasted good, though it was cold outside; implies that the cookie had some nutritional value to its owner when she baked it', 
'$answer$ = was a cheater and stole the cookies from his friend', 
'$answer$ = he overate them and put some in his mouth to finish the sentence.', 
"$answer$ = the cookies were harmed by his mother's carelessness and negligence, which led to him not being able to finish them all. rather than sweep up their footprints in an accident"], 

'generator_options': {'min_length': 1, 'max_length': 128, 'num_beams': 1, 'num_return_sequences': 5, 'do_sample': True, 'top_k': 10, 'top_p': 1.0, 'temperature': 1.0, 'length_penalty': 1.0, 'repetition_penalty': 3.0}, 'requested_angle': 'QC->AE', 

'input_slots': {'Question': 'What does the speaker imply by saying Alex ate some of the cookies?', 
'Context': 'The speaker is pragmatically savvy.'}, 

'output_slots_list': [{'answer': 'ate the cookies'}, 
{'answer': 'he must have enjoyed them. dessert tasted good, though it was cold outside; implies that the cookie had some nutritional value to its owner when she baked it'}, 
{'answer': 'was a cheater and stole the cookies from his friend'}, 
{'answer': 'he overate them and put some in his mouth to finish the sentence.'}, 
{'answer': "the cookies were harmed by his mother's carelessness and negligence, which led to him not being able to finish them all. rather than sweep up their footprints in an accident"}]}
'''

In [42]:
for i in df.index:
  q = df['question'][i]
  print('q: ', q)
  c = df['context'][i]
  print('c: ', c)
  results = question(q,c)
  df['answer'][i] = results['output_slots_list']

q:  What does the speaker imply by saying Alex is a good teacher?
c:  The speaker is pragmatically savvy.
q:  What does the speaker imply by saying it's warm outside?
c:  The speaker is pragmatically savvy.


In [43]:
df.head()

,condtion,target_adj,prediction,question,context,answer
0,SI,good,not excellent,What does the speaker imply by saying Alex is ...,The speaker is pragmatically savvy.,"[{'answer': 'she thinks so'}, {'answer': 'he k..."
1,SI,warm,not hot; not cold,What does the speaker imply by saying it's war...,The speaker is pragmatically savvy.,"[{'answer': 'it warms your heart'}, {'answer':..."


In [44]:
df['answer'][0]

[{'answer': 'she thinks so'},
 {'answer': 'he knows how to teach'},
 {'answer': 'he is good at explaining things'},
 {'answer': 'he is knowledgeable about math'},
 {'answer': 'that he is knowledgeable about math'}]

In [46]:
df.to_csv('si_adj.csv')

**Starting HERE: SI gradable adj**

In [64]:
df = pd.read_csv('si_adj.csv', index_col = 0)
df.head()

,condtion,target_adj,prediction,question_prag,question_sem,context_prag,context_sem,answer_prag_macaw,answer_sem_macaw,answer_prag_gpt,answer_sem_gpt
0,SI,good,not excellent; not bad,What does the speaker imply by saying Alex is ...,What does the speaker mean by saying Alex is a...,The speaker is pragmatically savvy.,The speaker is nerdy.,he has high expectations for his students. he ...,the class grew as Alex's knowledge of science ...,The speaker implies that Alex is a good teache...,The speaker is implying that Alex is a good te...
1,SI,warm,not hot; not cold,What does the speaker imply by saying it's war...,What does the speaker mean by saying it's warm...,The speaker is pragmatically savvy.,The speaker is nerdy.,it helps to warm the body. it warms your body....,the air temperature is higher than usual. it h...,The speaker is implying that it's warm outside...,The speaker is implying that it's warm outside...
2,MI,short,Pete is short,What does the speaker imply by saying Alex is ...,What does the speaker mean by saying Alex is a...,The speaker is pragmatically savvy.,The speaker is nerdy.,they have similar heights. he's as short for t...,the two men are alike in size. they have simil...,The speaker is implying that Alex is as short ...,The speaker implies that Alex is as short as P...


In [65]:
def question(question_str):
  res = run_macaw("Q: " + question_str +" \nA", model_dict,
                  {"do_sample": True, 'max_length': 128,
                   "top_k": 5, "num_return_sequences": 5, "repetition_penalty": 3.0})
  return res

In [66]:
for i in df.index:
  r_q = df['question_rett'][i]
  results = question(r_q)
  df['answer_rett_macaw'][i] = results['output_slots_list']
  temp_r = ''
  for r_dict in df['answer_rett_macaw'][i]:
    temp_r += r_dict['answer'] + '. '
    df['answer_prag_macaw'][i] = temp_r.strip(' ')

  p_q = df['question_prag'][i]
  results = question(p_q)
  df['answer_prag_macaw'][i] = results['output_slots_list']
  temp_p = ''
  for p_dict in df['answer_prag_macaw'][i]:
    temp_p += p_dict['answer'] + '. '
    df['answer_prag_macaw'][i] = temp_p.strip(' ')

  s_q = df['question_sem'][i]
  results = question(s_q)
  df['answer_sem_macaw'][i] = results['output_slots_list']
  temp_s = ''
  for s_dict in df['answer_sem_macaw'][i]:
    temp_s += s_dict['answer'] + '. '
    df['answer_sem_macaw'][i] = temp_s.strip(' ')

df.head()

,condtion,target_adj,prediction,question_prag,question_sem,context_prag,context_sem,answer_prag_macaw,answer_sem_macaw,answer_prag_gpt,answer_sem_gpt
0,SI,good,not excellent; not bad,What does the speaker imply by saying Alex is ...,What does the speaker mean by saying Alex is a...,The speaker is pragmatically savvy.,The speaker is nerdy.,he can teach other people. he has high expecta...,he's smart. he knows how to get the job done. ...,The speaker implies that Alex is a good teache...,The speaker is implying that Alex is a good te...
1,SI,warm,not hot; not cold,What does the speaker imply by saying it's war...,What does the speaker mean by saying it's warm...,The speaker is pragmatically savvy.,The speaker is nerdy.,it warms your body. the temperature is rising....,it's nicer than the air. it warms your hands. ...,The speaker is implying that it's warm outside...,The speaker is implying that it's warm outside...
2,MI,short,Pete is short,What does the speaker imply by saying Alex is ...,What does the speaker mean by saying Alex is a...,The speaker is pragmatically savvy.,The speaker is nerdy.,he has equal height. he has more hair than pet...,they have the same height. he is shorter than ...,The speaker is implying that Alex is as short ...,The speaker implies that Alex is as short as P...


In [67]:
df['answer_sem_macaw'][2]

"they have the same height. he is shorter than peter. is shorter than alex. he's short. he's shorter."

In [68]:
df['answer_prag_macaw'][2]

'he has equal height. he has more hair than peter. both are short in height. he is shorter than peter. he is short in stature.'

In [69]:
df.to_csv('si_adj.csv')